In [ ]:
import pandas as pd
import plotly.express as px

from autumn.core import inputs
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.utils.display import pretty_print
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes
from autumn.core.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline

### Population

In [ ]:
modelled_age_groups = baseline_params["age_groups"]
age_pops = pd.Series(
    inputs.get_population_by_agegroup(
        modelled_age_groups,
        baseline_params["country"]["iso3"], 
        baseline_params["population"]["region"],
        baseline_params["population"]["year"],
    ),
    index=modelled_age_groups
)
print(f"Total modelled population is {sum(age_pops) / 1e3} thousand")
print(f"Modelled age groups are {modelled_age_groups}")
age_pops.plot.bar()

### Mixing matrix

In [ ]:
print(f"Target country: {baseline_params['country']['iso3']}")
print(f"Target sub-region: {baseline_params['population']['region']}")
print(f"Proxy country: {baseline_params['ref_mixing_iso3']}")
print("Always age-adjusted to target population under current SM-SIR code")

In [ ]:
mixing_matrices = build_synthetic_matrices(
    baseline_params["country"]["iso3"],
    baseline_params["ref_mixing_iso3"],
    [int(age) for age in modelled_age_groups],
    True,
    baseline_params["population"]["region"],
)

In [ ]:
px.imshow(mixing_matrices["all_locations"])

### Calibration targets

In [ ]:
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [ ]:
ts_set_dates["notifications"].plot()